In [ ]:
!pip install transformers
!pip install torch
!pip install datasets

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import label_ranking_average_precision_score
import pandas as pd
from google.colab import drive
from datasets import Dataset
from transformers import BertForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import numpy as np
np.object = np.object_

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [3]:
#koBERT 모델 사용
model_name = "skt/kobert-base-v1"
tokenizer = KoBERTTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#하이퍼 파라미터 설정
MAX_LEN = 512  # 문장의 최대 길이를 512로 설정 (더 늘어나야 할 수도 있음)
BATCH_SIZE = 16  # 배치 크기를 줄여 더 세밀하게 학습
EPOCHS = 5  # 에폭 수
LEARNING_RATE = 2e-5  # 학습률 2e-5

In [5]:
unsmile_labels = ["여성/가족", "남성", "성소수자", "인종/국적", "연령", "지역", "종교", "악플/욕설", "clean"]

In [6]:
train_file_path = "/content/drive/MyDrive/Colab Notebooks/kakao-tech/final_team_pro/unsmile_merge.csv"
vaild_file_path = "/content/drive/MyDrive/Colab Notebooks/kakao-tech/final_team_pro/unsmile_valid_v1.0.tsv"

In [31]:
train_df = pd.read_csv(train_file_path, encoding='utf-8-sig')
valid_df = pd.read_csv(vaild_file_path, sep='\t', encoding='utf-8-sig')

In [27]:
train_df.head()

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


In [21]:
train_df.describe()

,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
count,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000,26113.000000
mean,0.063991,0.052120,0.047984,0.067131,0.024624,0.053039,0.046529,0.023054,0.120362,0.543178,0.012063
std,0.244742,0.222273,0.213736,0.250254,0.154979,0.224115,0.210631,0.150077,0.325390,0.498142,0.109169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
def preprocess_function(examples): #리벨링 전처리 코드
    # 토큰화 (여러 샘플에 대해 처리)
    tokenized_examples = tokenizer(
        examples["문장"],
        truncation=True,
        padding='max_length',  # max_length로 패딩
        max_length=MAX_LEN  # 시퀀스 길이를 설정 (512))
    )

    # 레이블 변환: 배치 단위로 처리
    labels = []
    # for example in examples["문장"]:  # 배치의 각 예제에 대해 레이블 리스트를 생성
    #     label = [examples[label] for label in unsmile_labels]  # 각 레이블 열에 대해 가져옴
    #     labels.append(label)  # 배치 내 각 예제에 대한 레이블을 추가

    # tokenized_examples['labels'] = labels  # 레이블을 tokenized_examples에 추가
    # return tokenized_examples
    for i in range(len(examples["문장"])):
        label = [examples[l][i] for l in unsmile_labels]  # 각 레이블을 리스트로 변환
        labels.append(label)

    tokenized_examples['labels'] = torch.tensor(labels, dtype=torch.float)  # float 타입으로 변환
    return tokenized_examples

In [9]:
# pandas DataFrame을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# 전처리 함수 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

# 텐서 형식으로 변환 (PyTorch 사용 시 필요)
train_dataset.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'labels', 'attention_mask'])

Map:   0%|          | 0/26113 [00:00<?, ? examples/s]

Map:   0%|          | 0/3737 [00:00<?, ? examples/s]

In [10]:
# DataLoader를 통해 데이터셋을 배치 단위로 처리
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
num_labels=len(unsmile_labels) # Label 갯수

model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)
model.config.id2label = {i: label for i, label in zip(range(num_labels), unsmile_labels)}
model.config.label2id = {label: i for i, label in zip(range(num_labels), unsmile_labels)}

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def compute_metrics(x):
    return {
        'lrap': label_ranking_average_precision_score(x.label_ids, x.predictions),
    }

In [14]:
# 저장할 경로를 지정
model_save_path = "/content/drive/MyDrive/Colab Notebooks/kakao-tech/final_team_pro/koBERT_train_fixed"

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [16]:
training_args = TrainingArguments(
    output_dir = model_save_path,  # 결과 저장 폴더
    evaluation_strategy="epoch",  # 매 에폭마다 평가
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='lrap',  # 가장 좋은 모델을 LRAP 기준으로 선택
    greater_is_better=True,
    gradient_accumulation_steps=2,  # 그래디언트 누적: 작은 배치로도 효과적 학습 가능
    weight_decay=0.01,  # 가중치 감소로 오버피팅 방지
    logging_dir='./logs',  # 로그 저장 폴더
    logging_steps=100,  # 로그 기록 빈도
    fp16=True  # 혼합 정밀도 학습으로 메모리 최적화 및 속도 향상
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    tokenizer=None, #koBERT 토크나이저 저장 오류 때문에 None처리
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
# 모델 학습
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss,Lrap
0,0.170900,0.243697,0.697832
2,0.086500,0.164264,0.835290
4,0.058800,0.173764,0.837881


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered i

TrainOutput(global_step=4080, training_loss=0.11340004007021585, metrics={'train_runtime': 694.3375, 'train_samples_per_second': 188.043, 'train_steps_per_second': 5.876, 'total_flos': 3.4338459036672e+16, 'train_loss': 0.11340004007021585, 'epoch': 4.996938150642988})

In [28]:
from sklearn.metrics import classification_report

def eval_model_with_report(model, data_loader, device):
    model = model.eval()  # 모델을 평가 모드로 설정
    predictions, targets = [], []

    with torch.no_grad():  # 평가 중에는 그래디언트를 계산하지 않음
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target = batch['labels'].to(device)  # 다중 레이블이므로 'labels' 사용

            # 모델의 출력 계산
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits)  # 시그모이드 적용하여 확률값으로 변환
            preds = (preds > 0.5).float()  # 임계값 0.5 이상이면 1로 예측

            predictions.extend(preds.cpu().numpy())  # CPU로 이동하여 리스트에 추가
            targets.extend(target.cpu().numpy())  # CPU로 이동하여 리스트에 추가

    # classification_report로 성능 평가 (다중 레이블 분류)
    report = classification_report(targets, predictions, target_names=unsmile_labels)
    return report

In [29]:
report = eval_model_with_report(model, valid_loader, device)
print(report)

              precision    recall  f1-score   support

       여성/가족       0.79      0.68      0.73       394
          남성       0.83      0.78      0.80       334
        성소수자       0.89      0.80      0.84       280
       인종/국적       0.83      0.73      0.78       426
          연령       0.86      0.69      0.77       146
          지역       0.88      0.78      0.83       260
          종교       0.88      0.83      0.86       290
       기타 혐오       1.00      0.04      0.09       134
       악플/욕설       0.62      0.54      0.58       786
       clean       0.68      0.72      0.70       935

   micro avg       0.76      0.68      0.72      3985
   macro avg       0.83      0.66      0.70      3985
weighted avg       0.77      0.68      0.71      3985
 samples avg       0.71      0.69      0.70      3985



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [32]:
# 모델 가중치와 설정 저장
model.save_pretrained(model_save_path)

# 토크나이저 설정 저장
tokenizer.save_pretrained(model_save_path)

TypeError: KoBertTokenizer.save_vocabulary() got an unexpected keyword argument 'filename_prefix'

In [34]:
import shutil
from google.colab import files

# 모델 디렉토리를 zip 파일로 압축
shutil.make_archive('KoBERT_model_v1.01', 'zip', 'KoBERT_model_v1.01')

# 생성된 zip 파일 다운로드
files.download('KoBERT_model_v1.01.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>